In [ ]:
!pip install -q efficientnet

In [ ]:
!pip install tensorflow_addons

In [ ]:
import math, re, os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
import efficientnet.tfkeras as efn
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications import DenseNet201
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print("Tensorflow version " + tf.__version__)
from tensorflow.keras import backend as K
import gc

In [ ]:
import tensorflow_addons as tfa

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

# Create strategy from tpu
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.experimental.TPUStrategy(tpu)

# Data access
GCS_DS_PATH = KaggleDatasets().get_gcs_path('flower-classification-with-tpus')

# Configuration
img_size=512
IMAGE_SIZE = [img_size, img_size]
EPOCHS = 20
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [ ]:
EPOCHS_ = 20
LR_START = 0.0001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync
LR_MIN = 0.000005
LR_RAMPUP_EPOCHS = 3
LR_SUSTAIN_EPOCHS = 3
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = np.random.random_sample() * LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(EPOCHS_)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
GCS_PATH_SELECT = { # available image sizes
    192: GCS_DS_PATH + '/tfrecords-jpeg-192x192',
    224: GCS_DS_PATH + '/tfrecords-jpeg-224x224',
    331: GCS_DS_PATH + '/tfrecords-jpeg-331x331',
    512: GCS_DS_PATH + '/tfrecords-jpeg-512x512'
}
GCS_PATH = GCS_PATH_SELECT[IMAGE_SIZE[0]]
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec') # predictions on this dataset should be submitted for the competition

# watch out for overfitting!
SKIP_VALIDATION = True
# import random
# random.shuffle(VALIDATION_FILENAMES)
# TRAINING_FILENAMES = TRAINING_FILENAMES+VALIDATION_FILENAMES[:8]
# VALIDATION_FILENAMES = VALIDATION_FILENAMES[8:]
# if SKIP_VALIDATION:
TRAINING_FILENAMES = TRAINING_FILENAMES + VALIDATION_FILENAMES

In [ ]:
SKIP_VALIDATION = True
import math, re, os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets

GCS_DS_PATH1 = KaggleDatasets().get_gcs_path('mytraindataset')
TRAINING_FILENAMES_ = tf.io.gfile.glob(GCS_DS_PATH1 + '/*.tfrec')

TRAINING_FILENAMES = TRAINING_FILENAMES + TRAINING_FILENAMES_

In [ ]:
print(GCS_DS_PATH1)

In [ ]:
len(TEST_FILENAMES)

In [ ]:
len(TRAINING_FILENAMES)

In [ ]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 102


In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.one_hot(example['class'], 104)
#     label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_labeled_tfrecord_train(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.one_hot(example['class'], 104)
#     label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) 
    dataset = dataset.with_options(ignore_order) 
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    return dataset

def load_dataset_train(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord_train if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def get_training_dataset(do_aug=False):
    dataset = load_dataset_train(TRAINING_FILENAMES, labeled=True)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    if do_aug: dataset = dataset.map(transform, num_parallel_calls=AUTO)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset



def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.random_flip_left_right(image)
    return image, label   

# def get_training_dataset():
#     dataset = load_dataset(TRAINING_FILENAMES, labeled=True)
#     dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
#     dataset = dataset.repeat() # the training dataset must repeat for several epochs
#     dataset = dataset.shuffle(2048)
#     dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
#     return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES, labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = (1 - SKIP_VALIDATION) * count_data_items(VALIDATION_FILENAMES)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE
print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))


In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

In [ ]:
def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 15. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3]),label

In [ ]:
test_ds = get_test_dataset(ordered=True)
test_images_ds = test_ds.map(lambda image, idnum: image)

In [ ]:
# Peek at training data
training_dataset = get_training_dataset()
training_dataset = training_dataset.unbatch().batch(20)
train_batch = iter(training_dataset)

In [ ]:
# data dump
print("Training data shapes:")
for image, label in get_training_dataset().take(3):
    print(image.numpy().shape, label.numpy().shape)
print("Training data label examples:", label.numpy())
print("Test data shapes:")
for image, idnum in get_test_dataset().take(3):
    print(image.numpy().shape, idnum.numpy().shape)
print("Test data IDs:", idnum.numpy().astype('U')) # U=unicode string

In [ ]:
from tensorflow.keras import backend as K

import dill


def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed


def categorical_focal_loss(gamma=2., alpha=.25):
    """
    Softmax version of focal loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Sum the losses in mini_batch
        return K.sum(loss, axis=1)

    return categorical_focal_loss_fixed


In [ ]:
# Learning rate schedule for TPU, GPU and CPU.
# Using an LR ramp up because fine-tuning a pre-trained model.
# Starting with a high LR would break the pre-trained weights.

LR_START = 0.00001
LR_MAX = 0.00005 * strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 4
LR_SUSTAIN_EPOCHS = 4
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

# rng = [i for i in range(25 if EPOCHS<25 else EPOCHS)]
# y = [lrfn(x) for x in rng]
# plt.plot(rng, y)
# print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
with strategy.scope():
    desnet=tf.keras.applications.Xception(
        weights='imagenet',
        include_top=False
      
    )
    model1 = tf.keras.Sequential([
        desnet,
        tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ]) 

model1.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
#     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
    loss = 'categorical_crossentropy',
    metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
)
model1.summary()
model1.load_weights('../input/mytopmodel/Xception_best.h5')

In [ ]:
with strategy.scope():
    desnet=tf.keras.applications.DenseNet201(
        weights='imagenet',
        include_top=False
      
    )
    model2 = tf.keras.Sequential([
        desnet,
        tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ]) 

model2.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
#     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
    loss = 'categorical_crossentropy',
    metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
)
model2.summary()
model2.load_weights('../input/mytopmodel/dense_best.h5')

In [ ]:
with strategy.scope():
    desnet=tf.keras.applications.InceptionResNetV2(
        weights='imagenet',
        include_top=False
      
    )
    model3 = tf.keras.Sequential([
        desnet,
        tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ]) 

model3.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
#     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
    loss = 'categorical_crossentropy',
    metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
)
model3.summary()
model3.load_weights('../input/mytopmodel/InceptionResV2_best.h5')

In [ ]:
with strategy.scope():
    desnet=tf.keras.applications.VGG19(
        weights='imagenet',
        include_top=False
      
    )
    model4 = tf.keras.Sequential([
        desnet,
        tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ]) 

model4.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
#     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
    loss = 'categorical_crossentropy',
    metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
)
model4.summary()
model4.load_weights('../input/mytopmodel/VGG_best.h5')

In [ ]:
with strategy.scope():
    desnet=efn.EfficientNetB5(
        weights='noisy-student',
        include_top=False
      
    )
    model5 = tf.keras.Sequential([
        desnet,
        tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ]) 

model5.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
#     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
    loss = 'categorical_crossentropy',
    metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
)
model5.summary()
model5.load_weights('../input/mytopmodel/B5_best.h5')

In [ ]:
with strategy.scope():
    desnet=tf.keras.applications.ResNet152V2(
        weights='imagenet',
        include_top=False
      
    )
    model6 = tf.keras.Sequential([
        desnet,
        tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax')
    ]) 

model6.compile(
    optimizer=tf.keras.optimizers.Adam(lr=0.0001),
#     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
    loss = 'categorical_crossentropy',
    metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
)
model6.summary()
model6.load_weights('../input/mytopmodel/Res152_best.h5')

In [ ]:
models = [model1, model2, model3, model4, model5, model6]

In [ ]:
test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.
test_images_ds = test_ds.map(lambda image, idnum: image)
print('Computing predictions...')
# get the mean probability of the folds models
probabilities = np.average([models[i].predict(test_images_ds) for i in range(6)], axis = 0)
predictions = np.argmax(probabilities, axis=-1)
print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
return histories, models

In [ ]:
# model11 = tf.keras.Sequential()
# for layer in model1.layers[:-2]:
#     model11.add(layer)
# for layer in model11.layers:
#     layer.trainable = False
# model22 = tf.keras.Sequential()
# for layer in model2.layers[:-2]:
#     model22.add(layer)
# for layer in model22.layers:
#     layer.trainable = False
    
# model33 = tf.keras.Sequential()
# for layer in model3.layers[:-2]:
#     model33.add(layer)
# for layer in model33.layers:
#     layer.trainable = False
    
    
# model44 = tf.keras.Sequential()
# for layer in model4.layers[:-2]:
#     model44.add(layer)
# for layer in model44.layers:
#     layer.trainable = False
    
# model55 = tf.keras.Sequential()
# for layer in model5.layers[:-2]:
#     model55.add(layer)
# for layer in model55.layers:
#     layer.trainable = False
    
# model66 = tf.keras.Sequential()
# for layer in model6.layers[:-2]:
#     model66.add(layer)
# for layer in model66.layers:
#     layer.trainable = False

In [ ]:
# def get_model():
#     with strategy.scope():
#         x = tf.keras.Input(shape = (img_size, img_size, 3))
#         x1 = model11(x)
#         x1 = tf.keras.layers.GlobalAveragePooling2D()(x1)
#         x2 = model22(x)
#         x2 = tf.keras.layers.GlobalAveragePooling2D()(x2)
#         x3 = model33(x)
#         x3 = tf.keras.layers.GlobalAveragePooling2D()(x3)
#         x4 = model44(x)
#         x4 = tf.keras.layers.GlobalAveragePooling2D()(x4)
#         x5 = model55(x)
#         x5 = tf.keras.layers.GlobalAveragePooling2D()(x5)
#         x6 = model66(x)
#         x6 = tf.keras.layers.GlobalAveragePooling2D()(x6)
#         x7 = tf.keras.layers.concatenate([x1, x2, x3, x4, x5, x6], axis = 1)
# #         x8 = tf.keras.layers.GlobalAveragePooling2D()(x7)
#         x9 = tf.keras.layers.Dropout(5/6)(x7)
#         x9 = tf.keras.layers.Dense(len(CLASSES), activation='softmax')(x9)
#         out = tf.keras.Model(inputs = x, outputs = x9)
#     out.compile(
#         optimizer=tf.keras.optimizers.Adam(lr=0.0001),
#     #     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
#         loss = 'categorical_crossentropy',
#         metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
#     )
#     return out
# # out.summary()
# # model4.load_weights('../input/mytopmodels/weights_efficient_23.h5')

In [ ]:
# my_call = tf.keras.callbacks.ModelCheckpoint('ensemble.h5', monitor='val_f1_score', mode='max', verbose=1, save_best_only=True, save_weights_only=True)

In [ ]:
# from sklearn.model_selection import KFold

In [ ]:
# FOLDS = 3
# SEED = 777
# import pandas as pd

In [ ]:
# def train_cross_validate(folds = 3):
#     histories = []
#     models = []
#     early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_f1_score', patience = 5, restore_best_weights=True)
#     kfold = KFold(folds, shuffle = True, random_state = SEED)
#     for f, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
#         print(); print('#'*25)
#         print('### FOLD',f+1)
#         print('#'*25)
#         train_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[trn_ind]['TRAINING_FILENAMES']), labeled = True)
#         val_dataset = load_dataset(list(pd.DataFrame({'TRAINING_FILENAMES': TRAINING_FILENAMES}).loc[val_ind]['TRAINING_FILENAMES']), labeled = True, ordered = True)
#         model = get_model()
#         history = model.fit(
#             get_training_dataset(train_dataset), 
#             steps_per_epoch = STEPS_PER_EPOCH,
#             epochs = EPOCHS,
#             callbacks = [lr_callback, early_stopping],
#             validation_data = get_validation_dataset(val_dataset),
#             verbose=2
#         )
#         models.append(model)
#         histories.append(history)
#     return histories, models

# def train_and_predict(folds = 3):
#     test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.
#     test_images_ds = test_ds.map(lambda image, idnum: image)
#     print('Start training %i folds'%folds)
#     histories, models = train_cross_validate(folds = folds)
#     print('Computing predictions...')
#     # get the mean probability of the folds models
#     probabilities = np.average([models[i].predict(test_images_ds) for i in range(folds)], axis = 0)
#     predictions = np.argmax(probabilities, axis=-1)
#     print('Generating submission.csv file...')
#     test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
#     test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
#     np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
#     return histories, models
    
# # run train and predict
# histories, models = train_and_predict(folds = FOLDS)

In [ ]:
# history = out.fit(
#     get_training_dataset(), 
#     steps_per_epoch=STEPS_PER_EPOCH,
#     epochs=EPOCHS,
#     callbacks=[lr_callback, my_call],
#     validation_data=None if SKIP_VALIDATION else get_validation_dataset(),
# #     validation_data = (val_images, val_labels),
#     shuffle = True
# )

In [ ]:
# with strategy.scope():
#     enet = efn.EfficientNetB4(
#         input_shape=(img_size, img_size, 3),
#         weights='noisy-student',
#         include_top=False
#     )

#     model2 = tf.keras.Sequential([
#         enet,
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ]) 

# model2.compile(
#     optimizer=tf.keras.optimizers.Adam(lr=0.0001),
#     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
# #     loss = 'categorical_crossentropy',
#     metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
# )
# model2.summary()
# model2.load_weights('../input/mytopmodels/weights_efficient_20.h5')

In [ ]:
# with strategy.scope():
#     enet = efn.EfficientNetB5(
#         input_shape=(img_size, img_size, 3),
#         weights='noisy-student',
#         include_top=False
#     )

#     model3 = tf.keras.Sequential([
#         enet,
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ]) 

# model3.compile(
#     optimizer=tf.keras.optimizers.Adam(lr=0.0001),
#     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
# #     loss = 'categorical_crossentropy',
#     metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
# )
# model3.summary()
# model3.load_weights('../input/mytopmodels/weights_efficient_24(1).h5')

In [ ]:
# with strategy.scope():
#     enet = efn.EfficientNetB6(
#         input_shape=(img_size, img_size, 3),
#         weights='noisy-student',
#         include_top=False
#     )

#     model4 = tf.keras.Sequential([
#         enet,
#         tf.keras.layers.GlobalAveragePooling2D(),
#         tf.keras.layers.Dense(len(CLASSES), activation='softmax')
#     ]) 

# model4.compile(
#     optimizer=tf.keras.optimizers.Adam(lr=0.0001),
#     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
# #     loss = 'categorical_crossentropy',
#     metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
# )
# model4.summary()
# model4.load_weights('../input/mytopmodels/weights_efficient_23.h5')

In [ ]:
# model11 = tf.keras.Sequential()
# for layer in model1.layers[:-2]:
#     model11.add(layer)
# for layer in model11.layers:
#     layer.trainable = False
# model22 = tf.keras.Sequential()
# for layer in model2.layers[:-2]:
#     model22.add(layer)
# for layer in model22.layers:
#     layer.trainable = False
    
# model33 = tf.keras.Sequential()
# for layer in model3.layers[:-2]:
#     model33.add(layer)
# for layer in model33.layers:
#     layer.trainable = False
    
    
# model44 = tf.keras.Sequential()
# for layer in model4.layers[:-2]:
#     model44.add(layer)
# for layer in model44.layers:
#     layer.trainable = False
    


In [ ]:
# with strategy.scope():
    
#     x = tf.keras.Input(shape = (img_size, img_size, 3))
#     x1 = model11(x)
#     x2 = model22(x)
#     x3 = model33(x)
#     x4 = model44(x)
#     x5 = tf.keras.layers.concatenate([x1, x2, x3, x4], axis = 3)
#     x6 = tf.keras.layers.GlobalAveragePooling2D()(x5)
#     x6 = tf.keras.layers.Dropout(0.75)(x6)
#     x6 = tf.keras.layers.Dense(len(CLASSES), activation='softmax')(x6)
#     out = tf.keras.Model(inputs = x, outputs = x6)

# out.compile(
#     optimizer=tf.keras.optimizers.Adam(lr=0.0001),
# #     loss = [categorical_focal_loss(gamma=2., alpha=.25)],
#     loss = 'categorical_crossentropy',
#     metrics=[tfa.metrics.F1Score(num_classes=104, average="macro")]
# )
# out.summary()
# # model4.load_weights('../input/mytopmodels/weights_efficient_23.h5')

In [ ]:
# EPOCHS_ = 50
# LR_START = 0.00001
# LR_MAX = 0.0004
# LR_MIN = 0.00001
# LR_RAMPUP_EPOCHS = 4
# LR_SUSTAIN_EPOCHS = 4
# LR_EXP_DECAY = .75
# test_ds = get_test_dataset(ordered=True)
# test_images_ds = test_ds.map(lambda image, idnum: image)
# n = 0
# def lrfn(epoch):
#     if epoch > 7:
#         out.save_weights("weights_ensemble_{}.h5".format(epoch))
#         print('Generating submission.csv file...')
#         test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
#         test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
#         np.savetxt("submission_ensemble_{}.csv".format(epoch), np.rec.fromarrays([test_ids, np.argmax(out.predict(test_images_ds), axis=-1)]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
#     if epoch - (epoch//25)*25 + 1 < LR_RAMPUP_EPOCHS:
#         lr = np.random.random_sample() * LR_START
#     elif epoch - (epoch//25)*25 + 1 < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
#         lr = LR_MAX
#     else:
#         lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - (epoch//25)*25 + 1 - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
#     return lr
    
# lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

# # rng = [i for i in range(EPOCHS_)]
# # y = [lrfn(x) for x in rng]
# # plt.plot(rng, y)
# # print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
# history = out.fit(
#     get_training_dataset(), 
#     steps_per_epoch=STEPS_PER_EPOCH,
#     epochs=EPOCHS,
#     callbacks=[lr_callback],
#     validation_data=None if SKIP_VALIDATION else get_validation_dataset(),
# #     validation_data = (val_images, val_labels),
#     shuffle = True
# )